In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
from datetime import datetime
import calendar

from config import db_user, db_password, db_host, db_port, db_name

In [2]:
old_data = pd.read_csv("static/data/Mother Jones - Mass Shootings Database, 1982 - 2021 - Sheet1.csv")

In [3]:
test = old_data["location"] 

In [4]:
city = []
state = []
for t in test:
    city.append(t.split(",")[0])
    state.append(t.split(", ")[1])

In [5]:
old_data["state"] = state
old_data["city"] = city
# spelling error in data
old_data["state"].replace({"Lousiana": "Louisiana"}, inplace=True)
old_data.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,gender,sources,mental_health_sources,sources_additional_age,latitude,longitude,type,year,state,city
0,San Jose VTA shooting,"San Jose, California",5/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,57,yes,...,M,https://www.sfchronicle.com/crime/article/Mass...,https://www.nytimes.com/2021/05/26/us/what-hap...,-,37.316097,-121.888533,Mass,2021,California,San Jose
1,FedEx warehouse shooting,"Indianapolis, Indiana",4/15/2021,"Brandon Scott Hole, 19, opened fire around 11 ...",8,7,15,Workplace,19,yes,...,M,https://www.indystar.com/story/news/crime/2021...,https://www.nytimes.com/2021/04/19/us/indianap...,-,39.686630,-86.323130,Mass,2021,Indiana,Indianapolis
2,Orange office complex shooting,"Orange, California",3/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,-,-,...,M,https://www.latimes.com/california/story/2021-...,-,-,33.835420,-117.853790,Mass,2021,California,Orange
3,Boulder supermarket shooting,"Boulder, Colorado",3/22/2021,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,0,10,Workplace,21,yes,...,M,https://www.denverpost.com/2021/03/22/police-a...,https://www.thedailybeast.com/boulder-colorado...,-,39.986961,-105.251168,Mass,2021,Colorado,Boulder
4,Atlanta massage parlor shootings,"Atlanta, Georgia",3/16/2021,"Robert Aaron Long, 21, killed eight people at ...",8,1,9,Workplace,21,-,...,M,https://www.ajc.com/news/breaking-multiple-sho...,-,-,34.111653,-84.580376,Spree,2021,Georgia,Atlanta


In [6]:
new_data = old_data[["case","location","date","summary","fatalities","injured","total_victims","location.1","age_of_shooter","prior_signs_mental_health_issues","mental_health_details","weapons_obtained_legally","where_obtained","race","gender","latitude","longitude","type","year","state","city"]]
new_data.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,weapons_obtained_legally,where_obtained,race,gender,latitude,longitude,type,year,state,city
0,San Jose VTA shooting,"San Jose, California",5/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,57,yes,...,-,-,-,M,37.316097,-121.888533,Mass,2021,California,San Jose
1,FedEx warehouse shooting,"Indianapolis, Indiana",4/15/2021,"Brandon Scott Hole, 19, opened fire around 11 ...",8,7,15,Workplace,19,yes,...,yes,-,White,M,39.686630,-86.323130,Mass,2021,Indiana,Indianapolis
2,Orange office complex shooting,"Orange, California",3/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,-,-,...,-,-,-,M,33.835420,-117.853790,Mass,2021,California,Orange
3,Boulder supermarket shooting,"Boulder, Colorado",3/22/2021,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,0,10,Workplace,21,yes,...,Yes,-,-,M,39.986961,-105.251168,Mass,2021,Colorado,Boulder
4,Atlanta massage parlor shootings,"Atlanta, Georgia",3/16/2021,"Robert Aaron Long, 21, killed eight people at ...",8,1,9,Workplace,21,-,...,-,-,White,M,34.111653,-84.580376,Spree,2021,Georgia,Atlanta


In [7]:
#Categorize the states into their respective regions and put them in a list
west = ["Colorado", "Wyoming", "Montana", "Idaho", "Washington", "Oregon",
                                  "Utah", "Nevada", "California", "Alaska", "Hawaii"]
southwest = ["Texas", "Oklahoma", "New Mexico", "Arizona"]
midwest = ["Ohio", "Indiana", "Michigan", "Illinois", "Missouri", "Wisconsin", "Minnesota",
                                     "Iowa", "Kansas", "Nebraska", "South Dakota", "North Dakota"]
southeast = ["West Virginia", "Virginia", "Kentucky", "Tennessee", "North Carolina",
                                       "South Carolina", "Georgia", "Alabama", "Mississippi", "Arkansas",
                                       "Louisiana", "Florida"] 
northeast = ["Maine", "Massachusetts", "Rhode Island", "Connecticut", "New Hampshire", "Maryland",
                                       "Vermont", "New York", "Pennsylvania", "New Jersey", "Delaware", "D.C."]

In [8]:
#Create a dataframe for each region
west_df = pd.DataFrame(west, columns = ["state"])
west_df['region'] = "West"

southwest_df = pd.DataFrame(southwest, columns = ["state"])
southwest_df['region'] = "Southwest"

midwest_df = pd.DataFrame(midwest, columns = ["state"])
midwest_df['region'] = "Midwest"

southeast_df = pd.DataFrame(southeast, columns = ["state"])
southeast_df['region'] = "Southeast"

northeast_df = pd.DataFrame(northeast, columns = ["state"])
northeast_df['region'] = "Northeast"

In [9]:
#Append diff region dataframes into one region indicator dataframe
region_df = west_df.copy()
region_df = region_df.append(southwest_df)
region_df = region_df.append(midwest_df)
region_df = region_df.append(southeast_df)
region_df = region_df.append(northeast_df)
region_df.head()

,state,region
0,Colorado,West
1,Wyoming,West
2,Montana,West
3,Idaho,West
4,Washington,West


In [10]:
#Merge the region & main dataframe together on 'state' column
merged_df = pd.merge(new_data, region_df, on='state')
merged_df.head()

,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,where_obtained,race,gender,latitude,longitude,type,year,state,city,region
0,San Jose VTA shooting,"San Jose, California",5/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,57,yes,...,-,-,M,37.316097,-121.888533,Mass,2021,California,San Jose,West
1,Orange office complex shooting,"Orange, California",3/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,-,-,...,-,-,M,33.835420,-117.853790,Mass,2021,California,Orange,West
2,Gilroy garlic festival shooting,"Gilroy, California",7/28/2019,"Santino William LeGan, 19, fired indiscriminat...",3,12,15,Other,19,TBD,...,"Nevada, on July 9",-,M,36.997191,-121.584819,Mass,2019,California,Gilroy,West
3,Thousand Oaks nightclub shooting,"Thousand Oaks, California",11/7/2018,"Ian David Long, 28, dressed in black and armed...",12,22,34,Other,28,Yes,...,A gun store in Simi Valley,-,M,34.176946,-118.874793,Mass,2018,California,Thousand Oaks,West
4,T&T Trucking shooting,"Bakersfield, California",9/12/2018,"Javier Casarez, 54, who was going through a bi...",5,0,5,Workplace,54,-,...,-,Latino,M,35.349388,-118.916335,Spree,2018,California,Bakersfield,West


In [11]:
#Change column order & remove first location column
new_merged_df = merged_df[["case","city", "state", "region", "date","summary","fatalities","injured","total_victims",
                     "location.1","age_of_shooter","prior_signs_mental_health_issues","mental_health_details",
                     "weapons_obtained_legally","race","gender","latitude","longitude","type","year"]]
new_merged_df

,case,city,state,region,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,mental_health_details,weapons_obtained_legally,race,gender,latitude,longitude,type,year
0,San Jose VTA shooting,San Jose,California,West,5/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,57,yes,"Perpetrator had a history of depression, angry...",-,-,M,37.316097,-121.888533,Mass,2021
1,Orange office complex shooting,Orange,California,West,3/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,-,-,-,-,-,M,33.835420,-117.853790,Mass,2021
2,Gilroy garlic festival shooting,Gilroy,California,West,7/28/2019,"Santino William LeGan, 19, fired indiscriminat...",3,12,15,Other,19,TBD,-,Yes,-,M,36.997191,-121.584819,Mass,2019
3,Thousand Oaks nightclub shooting,Thousand Oaks,California,West,11/7/2018,"Ian David Long, 28, dressed in black and armed...",12,22,34,Other,28,Yes,"Per the LATimes: ""Sheriff Geoff Dean said his ...",Yes,-,M,34.176946,-118.874793,Mass,2018
4,T&T Trucking shooting,Bakersfield,California,West,9/12/2018,"Javier Casarez, 54, who was going through a bi...",5,0,5,Workplace,54,-,-,Yes,Latino,M,35.349388,-118.916335,Spree,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Wakefield massacre,Wakefield,Massachusetts,Northeast,12/26/2000,"Michael McDermott, 42, opened fire on co-worke...",7,0,7,Workplace,42,Yes,"Psychiatrist guessed he had schizophrenia, but...",Yes,white,Male,42.500429,-71.075913,Mass,2000
120,Xerox killings,Honolulu,Hawaii,West,11/2/1999,"Byran Koji Uyesugi, 40, a Xerox service techni...",7,0,7,Workplace,40,Yes,"A psychiatrist, testifying for the prosecutio...",Yes,Asian,Male,21.320063,-157.876462,Spree,1999
121,Westside Middle School killings,Jonesboro,Arkansas,Southeast,3/24/1998,"Mitchell Scott Johnson, 13, and Andrew Douglas...",5,10,15,School,11,No,Boys displayed unruly and aggressive behavior....,No,white,Male,35.820990,-90.668261,Mass,1998
122,University of Iowa shooting,Iowa City,Iowa,Midwest,11/1/1991,"Former graduate student Gang Lu, 28, went on a...",6,1,7,School,28,Unclear,He was described as darkly disturbed and isola...,Yes,Asian,Male,41.660689,-91.530221,Spree,1991


In [12]:
#Change col name of location of shooting to "location"
new_merged_df = new_merged_df.rename(columns={'location.1':'location'})

### Clean up date

In [13]:
#Convert date column to a datetime object
new_merged_df['date'] = pd.to_datetime(new_merged_df['date'])

In [14]:
#Create a 'month' column by extracting the month from the date column
new_merged_df['month'] = pd.DatetimeIndex(new_merged_df['date']).month

#Change month number to abbreviation
new_merged_df['month'] = new_merged_df['month'].apply(lambda x: calendar.month_abbr[x])
new_merged_df.head()

,case,city,state,region,date,summary,fatalities,injured,total_victims,location,...,prior_signs_mental_health_issues,mental_health_details,weapons_obtained_legally,race,gender,latitude,longitude,type,year,month
0,San Jose VTA shooting,San Jose,California,West,2021-05-26,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,...,yes,"Perpetrator had a history of depression, angry...",-,-,M,37.316097,-121.888533,Mass,2021,May
1,Orange office complex shooting,Orange,California,West,2021-03-31,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,...,-,-,-,-,M,33.835420,-117.853790,Mass,2021,Mar
2,Gilroy garlic festival shooting,Gilroy,California,West,2019-07-28,"Santino William LeGan, 19, fired indiscriminat...",3,12,15,Other,...,TBD,-,Yes,-,M,36.997191,-121.584819,Mass,2019,Jul
3,Thousand Oaks nightclub shooting,Thousand Oaks,California,West,2018-11-07,"Ian David Long, 28, dressed in black and armed...",12,22,34,Other,...,Yes,"Per the LATimes: ""Sheriff Geoff Dean said his ...",Yes,-,M,34.176946,-118.874793,Mass,2018,Nov
4,T&T Trucking shooting,Bakersfield,California,West,2018-09-12,"Javier Casarez, 54, who was going through a bi...",5,0,5,Workplace,...,-,-,Yes,Latino,M,35.349388,-118.916335,Spree,2018,Sep


In [15]:
#Sort data by date in descending order
new_merged_df = new_merged_df.sort_values(by='date', ascending=False)

In [16]:
#Change format of date
new_merged_df['date'] = new_merged_df['date'].dt.strftime('%m/%d/%Y')
new_merged_df.head()

,case,city,state,region,date,summary,fatalities,injured,total_victims,location,...,prior_signs_mental_health_issues,mental_health_details,weapons_obtained_legally,race,gender,latitude,longitude,type,year,month
0,San Jose VTA shooting,San Jose,California,West,05/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,...,yes,"Perpetrator had a history of depression, angry...",-,-,M,37.316097,-121.888533,Mass,2021,May
22,FedEx warehouse shooting,Indianapolis,Indiana,Midwest,04/15/2021,"Brandon Scott Hole, 19, opened fire around 11 ...",8,7,15,Workplace,...,yes,Perpetrator had been reported to police in 202...,yes,White,M,39.686630,-86.323130,Mass,2021,Apr
1,Orange office complex shooting,Orange,California,West,03/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,...,-,-,-,-,M,33.835420,-117.853790,Mass,2021,Mar
23,Boulder supermarket shooting,Boulder,Colorado,West,03/22/2021,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,0,10,Workplace,...,yes,Brother described him as paranoid; multiple ac...,Yes,-,M,39.986961,-105.251168,Mass,2021,Mar
30,Atlanta massage parlor shootings,Atlanta,Georgia,Southeast,03/16/2021,"Robert Aaron Long, 21, killed eight people at ...",8,1,9,Workplace,...,-,-,-,White,M,34.111653,-84.580376,Spree,2021,Mar


### Update missing values to Unknown

In [17]:
#Pandas autofilled empty cells w/ a string containg \n. Replace any of those with "Unknown"
new_merged_df.loc[new_merged_df['location'].str.contains('\n'), 'location'] = 'Unknown'

#Update age_of_shooter column
new_merged_df['age_of_shooter'] = new_merged_df['age_of_shooter'].replace({'-': 'Unknown'})

#Update prior_signs_mental_health_issues column
new_merged_df['prior_signs_mental_health_issues'] = new_merged_df['prior_signs_mental_health_issues'].replace({'-': 'Unknown', 
                                                                                                              'TBD': 'Unknown', 'yes':'Yes', 'no': 'No'})

#Update mental_health_details column
new_merged_df['mental_health_details'] = new_merged_df['mental_health_details'].replace({'-': 'Unknown', '(pending)' : 'Unknown'})

#Update weapons_obtained_legally column
new_merged_df['weapons_obtained_legally'] = new_merged_df['weapons_obtained_legally'].replace({'-': 'Unknown', 'TBD' : 'Unknown',
                                           
                                                                                               'yes':'Yes', 'no': 'No'})
new_merged_df.loc[new_merged_df['weapons_obtained_legally'].str.contains('Yes'), 'weapons_obtained_legally'] = 'Yes'

#Replace the one last remaining data to Yes
new_merged_df.loc[(~new_merged_df['weapons_obtained_legally'].str.contains('Yes')) &
                      (~new_merged_df['weapons_obtained_legally'].str.contains('No')) &
                      (~new_merged_df['weapons_obtained_legally'].str.contains('Unknown')), 'weapons_obtained_legally'] = 'Yes'

#Update race column
new_merged_df['race'] = new_merged_df['race'].replace({'-': 'Unknown'}).str.title()

#Update gender column
new_merged_df['gender'] = new_merged_df['gender'].replace({'M': 'Male', 'F' : 'Female'})

new_merged_df.head()

,case,city,state,region,date,summary,fatalities,injured,total_victims,location,...,prior_signs_mental_health_issues,mental_health_details,weapons_obtained_legally,race,gender,latitude,longitude,type,year,month
0,San Jose VTA shooting,San Jose,California,West,05/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,...,Yes,"Perpetrator had a history of depression, angry...",Unknown,Unknown,Male,37.316097,-121.888533,Mass,2021,May
22,FedEx warehouse shooting,Indianapolis,Indiana,Midwest,04/15/2021,"Brandon Scott Hole, 19, opened fire around 11 ...",8,7,15,Workplace,...,Yes,Perpetrator had been reported to police in 202...,Yes,White,Male,39.686630,-86.323130,Mass,2021,Apr
1,Orange office complex shooting,Orange,California,West,03/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,...,Unknown,Unknown,Unknown,Unknown,Male,33.835420,-117.853790,Mass,2021,Mar
23,Boulder supermarket shooting,Boulder,Colorado,West,03/22/2021,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,0,10,Workplace,...,Yes,Brother described him as paranoid; multiple ac...,Yes,Unknown,Male,39.986961,-105.251168,Mass,2021,Mar
30,Atlanta massage parlor shootings,Atlanta,Georgia,Southeast,03/16/2021,"Robert Aaron Long, 21, killed eight people at ...",8,1,9,Workplace,...,Unknown,Unknown,Unknown,White,Male,34.111653,-84.580376,Spree,2021,Mar


In [18]:
#Rearrange columns
cleaned_df = new_merged_df[['case', 'city', 'state', 'region', 'date', 'summary', 'fatalities',
       'injured', 'total_victims', 'location', 'age_of_shooter',
       'prior_signs_mental_health_issues', 'mental_health_details',
       'weapons_obtained_legally', 'race', 'gender', 'latitude', 'longitude',
       'type','month', 'year']]
cleaned_df = cleaned_df.reset_index(drop=True)
cleaned_df.head()

,case,city,state,region,date,summary,fatalities,injured,total_victims,location,...,prior_signs_mental_health_issues,mental_health_details,weapons_obtained_legally,race,gender,latitude,longitude,type,month,year
0,San Jose VTA shooting,San Jose,California,West,05/26/2021,"Samuel Cassidy, 57, a Valley Transportation Au...",9,0,9,Workplace,...,Yes,"Perpetrator had a history of depression, angry...",Unknown,Unknown,Male,37.316097,-121.888533,Mass,May,2021
1,FedEx warehouse shooting,Indianapolis,Indiana,Midwest,04/15/2021,"Brandon Scott Hole, 19, opened fire around 11 ...",8,7,15,Workplace,...,Yes,Perpetrator had been reported to police in 202...,Yes,White,Male,39.686630,-86.323130,Mass,Apr,2021
2,Orange office complex shooting,Orange,California,West,03/31/2021,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,1,5,Workplace,...,Unknown,Unknown,Unknown,Unknown,Male,33.835420,-117.853790,Mass,Mar,2021
3,Boulder supermarket shooting,Boulder,Colorado,West,03/22/2021,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,0,10,Workplace,...,Yes,Brother described him as paranoid; multiple ac...,Yes,Unknown,Male,39.986961,-105.251168,Mass,Mar,2021
4,Atlanta massage parlor shootings,Atlanta,Georgia,Southeast,03/16/2021,"Robert Aaron Long, 21, killed eight people at ...",8,1,9,Workplace,...,Unknown,Unknown,Unknown,White,Male,34.111653,-84.580376,Spree,Mar,2021


### Load dataframe to postgres

In [19]:
#Configure the conection string
rds_connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

#Connect to the database
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [20]:
# Drop table is exists and append data
engine.execute('DROP TABLE IF EXISTS mass_shootings;')

cleaned_df.to_sql(name='mass_shootings', con=conn, if_exists='append', index=True)

In [21]:
try:
    engine.execute('ALTER TABLE mass_shootings ADD PRIMARY KEY (index);')
except Exception as e:
    print(e)